In [ ]:
pip install parlai

     |████████████████████████████████| 1.5 MB 4.2 MB/s 
     |████████████████████████████████| 547 kB 40.4 MB/s 
     |████████████████████████████████| 290 kB 52.7 MB/s 
     |████████████████████████████████| 235 kB 39.7 MB/s 
     |████████████████████████████████| 124 kB 49.3 MB/s 
     |████████████████████████████████| 3.3 MB 36.8 MB/s 
     |████████████████████████████████| 2.7 MB 21.6 MB/s 
     |████████████████████████████████| 170 kB 53.5 MB/s 
     |████████████████████████████████| 138 kB 49.9 MB/s 
     |████████████████████████████████| 74 kB 3.1 MB/s 
     |████████████████████████████████| 2.8 MB 20.6 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 48 kB 4.9 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 
     |████████████████████████████████| 46 kB 3.7 MB/s 
     |████████████████████████████████| 40 kB 5.5 MB/s 
     |███████████████████████

In [ ]:
import argparse
from parlai.utils.io import PathManager
from parlai.core.teachers import register_teacher, DialogTeacher
from parlai.scripts.display_model import DisplayModel
from parlai.scripts.train_model import TrainModel

In [ ]:
data = "parlai_combined.csv"

In [ ]:
@register_teacher("yo_teacher")
class YoTeacher(DialogTeacher):
    def __init__(self, opt, shared=None):
        self.datatype = opt['datatype']
        opt['datafile'] = data 
        self.id = 'yodialog'
        super().__init__(opt, shared)


    def setup_data(self, path):
    # note that path is the value provided by opt['datafile']
        prompt = ""                 # initialize prompt message
        cnter = 0                   # initialize conversation counter
        line_skipper = False        # to skip 1st row of curated dailog data with header: line
        new_episode = False
        print('loading: ' + path)
        with PathManager.open(path) as data_file:
            for row in data_file.readlines():
                if line_skipper:
                    row_array = row.rstrip('\n').split(';')
                    if cnter == 0:
                        prompt = row_array[1]
                        cnter += 1
                        new_episode = True if str.lower(row_array[0]) == 'true' else False
                    else:
                        yield {"text": prompt, "labels": row_array[1]}, new_episode
                        cnter = 0
                else:
                    line_skipper = True


In [ ]:
class DefaultTeacher(YoTeacher):
    pass

In [ ]:
TrainModel.main(
    # similar to before
    task='yo_teacher', 
    model='transformer/generator',
    model_file='from_pretrained/model_yo',
    
    # initialize with a pretrained model
    init_model='zoo:tutorial_transformer_generator/model',

    # BlenderBot 90M
    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
    dict_file='zoo:tutorial_transformer_generator/model.dict',
    learn_positional_embeddings=True,
    #dropout=0.1,
    #gradient_clip=0.1,
    #lr_scheduler='reduceonplateau',

    # some training arguments, specific to this fine-tuning
    # use a small learning rate with ADAM optimizer 1e-5,
    lr=1e-05, optimizer='adam',
    warmup_updates=100,
    # early stopping on perplexity
    validation_metric='ppl',
    # train at most 10 minutes, and validate every 0.25 epochs
    max_train_time=20 * 60, validation_every_n_epochs=0.25,
    
    # depend on your gpu. If you have a V100, this is good
    batchsize=6, fp16=True, fp16_impl='mem_efficient',
    
    # speeds up validation
    skip_generation=False,
    
    # helps us cram more examples into our gpu at a time
    dynamic_batching='full',
)


12:20:42 | building dictionary first...
12:20:42 | No model with opt yet at: from_pretrained/model_yo(.opt)
12:20:42 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: allow_missing_init_opts: False,download_path: None,loglevel: info,dynamic_batching: full,verbose: False,is_debug: False,datapath: /usr/local/lib/python3.7/dist-packages/data,final_extra_opt: ,eval_dynamic_batching: None,num_workers: 0,max_train_steps: -1,log_every_n_steps: 50,validation_every_n_steps: -1,load_from_checkpoint: True,tensorboard_logdir: None,wandb_log: False,wandb_name: None,wandb_project: None,wandb_entity: None,mutators: None,n_encoder_layers: -1,n_decoder_layers: -1,model_parallel: False,checkpoint_activations: False,beam_block_full_context: True,beam_length_penalty: 0.65,topk: 10,topp: 0.9,beam_delay: 30,beam_block_list_filename: None,temperature: 1.0,compute_tokenized_bleu: False,interactive_mode: False,fp16_impl: mem_efficient,force_fp16_tokens

({'accuracy': ExactMatchMetric(0),
  'bleu-4': BleuMetric(0.0001246),
  'clen': AverageMetric(7174),
  'ctpb': GlobalAverageMetric(951.1),
  'ctps': GlobalTimerMetric(337),
  'ctrunc': AverageMetric(0.942),
  'ctrunclen': AverageMetric(6678),
  'exps': GlobalTimerMetric(0.6791),
  'exs': SumMetric(759),
  'f1': F1Metric(0.03009),
  'gpu_mem': GlobalAverageMetric(0.2449),
  'llen': AverageMetric(27),
  'loss': AverageMetric(5.041),
  'lr': GlobalAverageMetric(2.01e-07),
  'ltpb': GlobalAverageMetric(51.29),
  'ltps': GlobalTimerMetric(18.17),
  'ltrunc': AverageMetric(0.003953),
  'ltrunclen': AverageMetric(0.2411),
  'ppl': PPLMetric(154.6),
  'token_acc': AverageMetric(0.2525),
  'token_em': AverageMetric(0),
  'total_train_updates': GlobalFixedMetric(2),
  'tpb': GlobalAverageMetric(1002),
  'tps': GlobalTimerMetric(355.2)},
 {'accuracy': ExactMatchMetric(0),
  'bleu-4': BleuMetric(0.0001246),
  'clen': AverageMetric(7174),
  'ctpb': GlobalAverageMetric(951.1),
  'ctps': GlobalTimerM

In [ ]:
DisplayModel.main(task='yo_teacher', model_file='from_pretrained/model_yo', num_examples=3, skip_generation=False)

13:37:31 | Using CUDA
13:37:31 | loading dictionary from from_pretrained/model_yo.dict
13:37:32 | num words = 54944
13:37:33 | Total parameters: 87,508,992 (87,508,992 trainable)
13:37:33 | Loading existing model params from from_pretrained/model_yo
13:37:36 | creating task(s): yo_teacher
loading: parlai_combined.csv
13:37:36 | Opt:
13:37:36 |     activation: gelu
13:37:36 |     adafactor_eps: '[1e-30, 0.001]'
13:37:36 |     adam_eps: 1e-08
13:37:36 |     add_p1_after_newln: False
13:37:36 |     aggregate_micro: False
13:37:36 |     allow_missing_init_opts: False
13:37:36 |     attention_dropout: 0.0
13:37:36 |     batchsize: 6
13:37:36 |     beam_block_full_context: True
13:37:36 |     beam_block_list_filename: None
13:37:36 |     beam_block_ngram: -1
13:37:36 |     beam_context_block_ngram: -1
13:37:36 |     beam_delay: 30
13:37:36 |     beam_length_penalty: 0.65
13:37:36 |     beam_min_length: 1
13:37:36 |     beam_size: 1
13:37:36 |     betas: '[0.9, 0.999]'
13:37:36 |     bpe_add_

**Please provide a brief assessment of the 3 conversations above here. What do you think about the generated text? Is it repetitive, meaningless, etc?** Example below:

We observe from the first example that the model sometimes fails to respond in the target language. And the last two examples show that the model can be repetitively bad.